<a href="https://colab.research.google.com/github/RangaSudharshan05/Sequence-Analysis/blob/main/protein_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install biopython==1.75

     |████████████████████████████████| 2.3MB 4.2MB/s 
  Found existing installation: biopython 1.68
    Uninstalling biopython-1.68:
      Successfully uninstalled biopython-1.68


In [55]:
import Bio
import nglview as nv
import pandas as pd
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio.Alphabet import generic_dna, generic_rna,generic_protein, IUPAC
from collections import Counter

In [17]:
dir(Bio.Data.CodonTable)

['Alphabet',
 'AmbiguousCodonTable',
 'AmbiguousForwardTable',
 'CodonTable',
 'IUPAC',
 'IUPACData',
 'NCBICodonTable',
 'NCBICodonTableDNA',
 'NCBICodonTableRNA',
 'TranslationError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'ambiguous_dna_by_id',
 'ambiguous_dna_by_name',
 'ambiguous_generic_by_id',
 'ambiguous_generic_by_name',
 'ambiguous_rna_by_id',
 'ambiguous_rna_by_name',
 'generic_by_id',
 'generic_by_name',
 'list_ambiguous_codons',
 'list_possible_proteins',
 'make_back_table',
 'register_ncbi_table',
 'standard_dna_table',
 'standard_rna_table',
 'unambiguous_dna_by_id',
 'unambiguous_dna_by_name',
 'unambiguous_rna_by_id',
 'unambiguous_rna_by_name']

Reading the FASTA file of SARS-COV-2 and checking the ID with the genbank

In [29]:
from Bio import SeqIO
for i in SeqIO.parse('sequence.fasta','fasta'):
  print(i )

ID: MN908947.3
Name: MN908947.3
Description: MN908947.3 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())


In [32]:
covid_dna=SeqIO.read('sequence.fasta','fasta').seq
print('The length of the covid-19 dna is', len(covid_dna))
covid_dna

The length of the covid-19 dna is 29903


Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())

In [36]:
#transcribing and translating the covd dna
covid_mrna=covid_dna.transcribe()
covid_protein=covid_mrna.translate()
print('Number of proteins after translating the covid dna is', len(covid_protein))

Number of proteins after translating the covid dna is 9967


/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2748: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [38]:
#finding out proteins by the spliting te sequence where the stop codon is present and coverting the data into dataframe
covid_aa=covid_protein.split('*')
covid_aa=[str(i) for i in covid_aa]
covid_aa=pd.DataFrame({'Amino Acids':covid_aa})
covid_aa

,Amino Acids
0,IKGLYLPR
1,QTNQLSISCRSVL
2,TNFKICVAVTRLHA
3,CTHAV
4,LITNYCR
...,...
770,SHIAIFNQCVTLGRT
771,KSHHIFTEATRSTIECTVNNARESCLYGRALMCKINFSSAIPM
772,F
773,


In [42]:
covid_aa['Count']=covid_aa['Amino Acids'].str.len()
covid_aa.head()

,Amino Acids,Count
0,IKGLYLPR,8
1,QTNQLSISCRSVL,13
2,TNFKICVAVTRLHA,14
3,CTHAV,5
4,LITNYCR,7


In [45]:
covid_aa.Count.sort_values(ascending=False)

548    2701
694     290
719     123
695      83
718      63
       ... 
98        0
101       0
103       0
105       0
566       0
Name: Count, Length: 775, dtype: int64

In [46]:
#largest aminoacid after translating the mrna from the genome
covid_aa['Amino Acids'][548]

'CTIVFKRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSY

In [51]:
#5 most common amino acid
Counter(covid_protein).most_common(5)

[('L', 886), ('S', 810), ('*', 774), ('T', 679), ('C', 635)]

In [52]:
!pip install nglview

     |████████████████████████████████| 5.2MB 4.2MB/s 
  Created wheel for nglview: filename=nglview-2.7.7-cp36-none-any.whl size=6983523 sha256=9812474afeb4a7adb583659cf4079a34329b07cf29c46193acfe0b89d9fa30b2
  Stored in directory: /root/.cache/pip/wheels/24/14/56/fc0406301850d9d017a748b568b2e97566148deda046ac56dc
Successfully built nglview


In [56]:
#visualising the 3 D structure
parser= PDBParser()
structure=parser.get_structure('mmdb_6LU7','mmdb_6LU7.pdb')

In [60]:
nv.show_biopython(structure)._display_image()

In [61]:
!pip install pytraj

     |████████████████████████████████| 16.1MB 1.7MB/s 


In [62]:
import pytraj

In [63]:
covid_view=pytraj.load('mmdb_6LU7.pdb')

In [64]:
nv.show_pytraj(covid_view)._display_image()